# 🎬 Tech Challenge Fase 4 - Upload Manual

## 📋 Como usar este notebook:
1. ⚡ **Ative a GPU**: Runtime → Change runtime type → GPU (T4)
2. 📤 **Faça upload** da pasta `src/` (ou do ZIP do projeto)
3. ▶️ **Execute** todas as células
4. 🎥 **Upload** do seu vídeo quando solicitado
5. ⬇️ **Baixe** os resultados

## ⚠️ IMPORTANTE:
Este notebook requer upload manual dos arquivos do projeto.

**Opção mais fácil**: Use o notebook principal com Git clone.

## 1️⃣ Verificar GPU e Recursos

In [ ]:
# Verificar GPU
!nvidia-smi -L

# Verificar recursos
import psutil
print(f"\n💾 RAM Total: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"💾 RAM Disponível: {psutil.virtual_memory().available / (1024**3):.1f} GB")
print(f"🖥️  CPU Cores: {psutil.cpu_count()}")

## 2️⃣ Instalar Dependências

In [ ]:
# Criar estrutura de diretórios do projeto
import os

directories = [
    'outputs',
    'outputs/logs', 
    'outputs/frames',
    'data',
    'data/input_video',
    'models'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)

print("✅ Estrutura de diretórios criada!")

In [ ]:
!pip install -q opencv-python numpy pandas deepface torch torchvision tqdm pydantic rich moviepy
print("✅ Dependências instaladas!")

## 3️⃣ Upload dos Arquivos do Projeto

**Opção A: Upload do ZIP completo**

In [ ]:
from google.colab import files
import zipfile
import os

print("📤 Faça upload do arquivo ZIP do projeto:")
print("   (TechChallengeFase4.zip)")
uploaded = files.upload()

# Descompactar
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Descompactando {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Projeto descompactado!")

# Listar estrutura
print("\n📂 Estrutura do projeto:")
!ls -la

**Opção B: Criar estrutura manualmente** (se não tiver ZIP)

In [ ]:
# Criar estrutura de diretórios
import os

dirs = [
    'src',
    'src/io',
    'src/face',
    'src/emotion',
    'src/activity',
    'src/pipeline',
    'src/metrics',
    'src/utils',
    'outputs',
    'outputs/logs',
    'outputs/frames'
]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    # Criar __init__.py
    if d.startswith('src'):
        with open(f"{d}/__init__.py", 'w') as f:
            f.write("")

print("✅ Estrutura de diretórios criada!")
print("\n⚠️ AGORA: Faça upload dos arquivos .py para cada pasta correspondente")
print("   Use o painel lateral do Colab para fazer upload dos arquivos")

## 4️⃣ Configurar Python Path

In [ ]:
import sys
import os

# Encontrar diretório do projeto
project_dirs = ['/content', '/content/TechChallengeFase4']
project_dir = None

for d in project_dirs:
    if os.path.exists(os.path.join(d, 'src')):
        project_dir = d
        break

if project_dir:
    os.chdir(project_dir)
    if project_dir not in sys.path:
        sys.path.insert(0, project_dir)
    print(f"✅ Diretório do projeto: {project_dir}")
    print(f"✅ Python path configurado!")
    
    # Verificar estrutura
    print("\n📂 Verificando arquivos:")
    !ls -la src/
else:
    print("❌ Erro: Pasta 'src' não encontrada!")
    print("💡 Certifique-se que fez upload do projeto corretamente")

## 5️⃣ Testar Importações

In [ ]:
# Testar importações
try:
    from src.pipeline.inference import InferencePipeline
    from src.metrics.reporter import Reporter
    from src.io.video_reader import VideoReader
    from src.face.detector import FaceDetector
    
    print("✅ Todas as importações funcionaram!")
    print("🎬 Sistema pronto para processar vídeos!\n")
    
except ImportError as e:
    print(f"❌ Erro ao importar: {e}")
    print("\n🔍 Arquivos disponíveis em src/:")
    !find src -name "*.py" -type f
    print("\n💡 Solução:")
    print("   1. Certifique-se que todos os arquivos .py foram carregados")
    print("   2. Verifique se os __init__.py existem em cada pasta")
    print("   3. Execute a célula anterior novamente")

## 6️⃣ Upload do Vídeo para Análise

In [ ]:
from google.colab import files

print("📤 Faça upload do seu vídeo:")
uploaded = files.upload()

video_filename = list(uploaded.keys())[0]
print(f"\n✅ Vídeo carregado: {video_filename}")
print(f"📊 Tamanho: {len(uploaded[video_filename]) / (1024**2):.2f} MB")

## 7️⃣ Processar Vídeo

In [ ]:
from src.pipeline.inference import InferencePipeline

print("🎬 Iniciando processamento...\n")

# Criar pipeline
pipeline = InferencePipeline(
    video_path=video_filename,
    output_video_path="outputs/annotated_video.mp4",
    save_preview=True,
    face_backend="opencv",
    emotion_backend="deepface"
)

# Processar
summary = pipeline.run()

print("\n✅ Processamento concluído!")

## 8️⃣ Visualizar Resultados

In [ ]:
from IPython.display import Video

# Mostrar métricas
print("📊 RESUMO")
print("=" * 50)
print(f"📹 Vídeo: {summary['video_path']}")
print(f"🎞️  Frames: {summary['frames_total']:,}")
print(f"⏱️  Duração: {summary['duration_seconds']:.2f}s")
print(f"\n👤 Faces: {summary['faces_stats']['total']}")
print(f"\n⚠️  Anomalias: {summary['anomalies_total']}")

# Mostrar vídeo
print("\n🎬 Vídeo Anotado:")
Video("outputs/annotated_video.mp4", width=800)

## 9️⃣ Download dos Resultados

In [ ]:
from google.colab import files
from src.metrics.reporter import Reporter
import zipfile

# Gerar relatórios
reporter = Reporter()
reporter.save_metrics(summary, "outputs/metrics.json")
reporter.generate_markdown_report(summary, "outputs/report.md")

# Criar ZIP
with zipfile.ZipFile("results.zip", 'w') as zipf:
    zipf.write("outputs/annotated_video.mp4", "annotated_video.mp4")
    zipf.write("outputs/metrics.json", "metrics.json")
    zipf.write("outputs/report.md", "report.md")

print("📦 Baixando resultados...")
files.download("results.zip")